In [2]:
using LinearAlgebra
using ControlSystems
using Plots
using ControlSystemIdentification
using Random
using Statistics
using DelimitedFiles
using CUDA
using Flux
using Hyperopt
using FluxArchitectures


include("KugaPack.jl")
include("sysid.jl")
include("blockpack.jl")
include("analysis.jl")

n4sidPatternArg (generic function with 1 method)

In [3]:
#numerical conditions are set up
dt = 1e-3;
maxT = 7;
ns = round(Int,maxT/dt)+1;
t = 0:dt:maxT;

ns

7001

In [4]:
x0 = [0; 0; 0; 0;];

n_frac = round(Int,ns/4)

w = 80
U1 = chirp(w,0:dt:maxT/4-dt)
U2 = PulseGen(n_frac+n_frac+1)
U3 = rand(1,n_frac).-0.5;


U = vec([U1; U2'; U3']);

In [5]:
#using the given parameters, the system (discrete space state) is created 
Param = [0.77 0.59 2.1 1.2 9 200 200 200];
sysP,sysD = F2DOF(dt,Param);

Y,_,_= lsim(sysD,U',t,x0=x0); #time response
magD,phaseD,w = bode(sysD); #frequency response

In [6]:
#identification parameters
na = 4;
nb = 1;
ny = 2;
nu = 1;
p = 25;

In [7]:
Data = DataNARX(DataNorm(Y),U', ny, nu, na, nb,ns)
dataSize = ny*na+nb*nu;

model = TrainModel(
    NNARX_LSTM(dataSize,p,ny),
    Data
    )


Chain(
  Recur(
    LSTMCell(9 => 9),                   # 702 parameters
  ),
  Recur(
    LSTMCell(9 => 1),                   # 46 parameters
  ),
  Dense(1 => 25, tanh),                 # 50 parameters
  Dense(25 => 2),                       # 52 parameters
)         # Total: 14 trainable arrays, 850 parameters,
          # plus 4 non-trainable, 20 parameters, summarysize 2.469 KiB.

In [10]:
model([U[1] ; zeros(ny*na)] |> gpu)

Hn = zeros(na*ny+nb*nu,ns)



1